In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms 

import warnings
warnings.filterwarnings('ignore')

In [3]:
import timm
import torch
import torchvision.transforms as transforms

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

def get_models(dataset, model_name, key):
    """
    Creates a model for CIFAR-10 or ImageNet dataset.
    For CIFAR-10: uses pretrained ImageNet weights, replaces classifier head with num_classes=10.
    """
    if dataset == 'imagenet':
        model = timm.create_model(model_name, pretrained=True, num_classes=1000).to(device)
        model.eval()
        if any(x in key for x in ['inc', 'vit', 'bit']):
            norm = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        else:
            norm = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        return torch.nn.Sequential(norm, model)
    
    if dataset == 'cifar100':
        # For Inception, resize to 299x299; otherwise 32x32 is fine
        if any(x in key for x in ['inc', 'vit', 'bit']):
            transform_resize = transforms.Resize((299, 299))
            norm = transforms.Normalize((0.5,), (0.5,)) 
            
        else:
            transform_resize = transforms.Resize((224, 224)) # do nothing
            # Standard CIFAR-10 normalization
            norm = transforms.Normalize((0.4914, 0.4822, 0.4465),
                                        (0.2023, 0.1994, 0.2010)) 
    
        # Create model
        model = timm.create_model(model_name, pretrained=True, num_classes=100).to(device)
        model.eval()
        
        # Wrap resize + normalization + model in Sequential
        return torch.nn.Sequential(
            transform_resize,   # resize if Inception
            norm,               # normalization
            model
        )

In [4]:
import torchvision
from torch.utils.data import random_split, DataLoader

# CIFAR-10 normalization for training and test
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465),
    #                      (0.2023, 0.1994, 0.2010)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465),
    #                      (0.2023, 0.1994, 0.2010)),
])

# Load datasets
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

# Split train into train/val
train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
train_subset, val_subset = random_split(trainset, [train_size, val_size], generator=torch.Generator().manual_seed(56))

# DataLoaders
trainloader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
valloader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=4)

print(f"Train: {len(train_subset)}, Val: {len(val_subset)}, Test: {len(testset)}")

Train: 45000, Val: 5000, Test: 10000


### Train 

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def train_model(
    model,
    trainloader,
    valloader,
    epochs=10,
    lr=1e-3,
    checkpoint_path="best_model.pth",
    early_stop_patience=None,
):

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4 )
    scaler = torch.cuda.amp.GradScaler()     # Mixed precision

    best_val_acc = 0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss, total, correct = 0, 0, 0

        pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}")

        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():   # FP16 forward
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix(loss=loss.item())

        train_acc = 100 * correct / total
        val_acc = evaluate(model, valloader)

        print(
            f"Epoch {epoch+1}: "
            f"Loss={running_loss/total:.4f}, "
            f"Train Acc={train_acc:.2f}%, "
            f"Val Acc={val_acc:.2f}%"
        )

        # -------------------------------
        # Save Best Model Checkpoint
        # -------------------------------
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(
                {
                    "epoch": epoch + 1,
                    "model_state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "val_acc": best_val_acc,
                },
                checkpoint_path,
            )
            print(f"✨ Saved Best Model (Val Acc: {best_val_acc:.2f}%)")

            patience_counter = 0
        else:
            patience_counter += 1

        # -------------------------------
        # Early Stopping
        # -------------------------------
        if early_stop_patience is not None:
            if patience_counter >= early_stop_patience:
                print("⛔ Early stopping triggered.")
                break

    print(f"Training completed. Best Val Acc: {best_val_acc:.2f}%")
    return best_val_acc


In [6]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += predicted.eq(y).sum().item()
    return 100 * correct / total

In [7]:
model_name = "resnetv2_101x1_bitm" # regnety_160, efficientvit_b0, convmixer_768_32  

model = get_models('cifar100', model_name, "resnetv2_101x1_bitm") 

train_model(model, trainloader, valloader, epochs=100, lr=0.005, checkpoint_path=f"checkpoints/{model_name}_cifar100.pth", 
           early_stop_patience=None)

Epoch 1/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1407/1407 [07:06<00:00,  3.30it/s, loss=0.487]


Epoch 1: Loss=1.6514, Train Acc=55.36%, Val Acc=63.02%
✨ Saved Best Model (Val Acc: 63.02%)


Epoch 2/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1407/1407 [07:03<00:00,  3.32it/s, loss=0.891]


Epoch 2: Loss=0.8985, Train Acc=73.21%, Val Acc=70.58%
✨ Saved Best Model (Val Acc: 70.58%)


Epoch 3/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1407/1407 [07:03<00:00,  3.32it/s, loss=1.4]


...

Epoch 84: Loss=0.2105, Train Acc=93.67%, Val Acc=74.06%


Epoch 85/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1407/1407 [07:04<00:00,  3.31it/s, loss=0.0592]


Epoch 85: Loss=0.2018, Train Acc=93.95%, Val Acc=75.42%


Epoch 86/100:  23%|█████████████████████████████                                                                                               | 330/1407 [01:40<05:27,  3.29it/s, loss=0.107]


KeyboardInterrupt: 